<a href="https://colab.research.google.com/github/Jorgepratesn/Projetos/blob/main/Projeto_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Insights House Rocket

## Contexto do Desafio:

A House Rocket é uma plataforma digital que tem como modelo de negócio, a compra e a venda de imóveis usando tecnologia.

O CEO da House Rocket gostaria de maximizar a receita da empresa encontrando boas oportunidades de negócio.

Sua principal estratégia é comprar boas casas em ótimas localizações com preços baixos e depois revendê-las posteriormente à preços mais altos. Quanto maior a diferença entre a compra e a venda, maior o lucro da empresa e portanto maior sua receita.

Entretanto, as casas possuem muitos atributos que as tornam mais ou menos atrativas aos compradores e vendedores e a localização e o período do ano também podem influenciar os preços.

O projeto desenvolvido a seguir tem como objetivo responder 3 perguntas:

1.   Quais casas o CEO da House Rocket deveria comprar e por qual preço de compra?
2.   Uma vez a casa em posse da empresa, qual o melhor momento para vendê-las e qual seria o preço da venda?
3.   A House Rocket deveria fazer uma reforma para aumentar o preço da venda? Quais seriam as sugestões de mudanças? Qual o incremento no preço dado por cada opção de reforma?

## Importação de pacotes basicos

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Base de dados



*   Fonte dos dados: https://www.kaggle.com/datasets/harlfoxem/housesalesprediction?resource=download




In [66]:
base_house=pd.read_csv('https://raw.githubusercontent.com/Jorgepratesn/Projetos/main/kc_house_data.csv')

## Exploração e limpeza dos Dados

In [ ]:
#Verificando as variaveis presentes na tabela
base_house.head(10)

In [4]:
#Alterando a variavel yr_renovated para um binario entre renovada ou não
base_house['renovated'] = base_house['yr_renovated'].apply(lambda x: '0' if x == 0 else '1')

In [5]:
#Excluindo variaveis que não serão usadas na análise
base_house = base_house.drop(['lat', 'long','yr_renovated'], axis=1)

In [ ]:
#Verificar os tipos de variáveis na base de dados
base_house.dtypes

In [ ]:
#Alterar os tipos de variaveis que estão errados
base_house['id'] = base_house['id'].astype('category')
base_house['waterfront'] = base_house['waterfront'].astype('category')
base_house['condition'] = base_house['condition'].astype('category')
base_house['zipcode'] = base_house['zipcode'].astype('category')
base_house.dtypes

In [ ]:
#Verificação para entender se será necessario tratar valores faltantes
base_house.isnull().sum()

In [ ]:
#Estatisticas descritivas das variaveis qualitativas - Entendendo como estão distribuidos os dados
#Por exemplo: É possivel observar que a grande maioria das cadas tem 3 ou mais banheiros e tem entre 1 e 2 pavimentos
#Pode-se observar tambem que não há valores negativos na base
variaveis_quantitativas = base_house.select_dtypes(include='number')
variaveis_quantitativas.describe()

In [ ]:
#Para as variaveis quantitativas podemos entender quais são as caracteristicas mais presentes
#Por exemplo: É possivel observar que 99,2% das casas não são de frente para o mar
base_house["waterfront"].value_counts()
base_house["zipcode"].value_counts()
base_house["renovated"].value_counts()
base_house["condition"].value_counts()

In [ ]:
#Graficos boxplot para definição de outliers

# Definir as variaveis que serão utilizadas
variaveis_categoricas = ['waterfront', 'bedrooms', 'bathrooms', 'floors', 'condition', 'grade', 'yr_built','zipcode']
Y1 = base_house["price"]
# Configurar o tamanho da figura
plt.figure(figsize=(14, 8))
# Criar um subplot com várias boxplots
for i, variavel in enumerate(variaveis_categoricas, 1):
    plt.subplot(2, 4, i)
    sns.boxplot(x=variavel, y=Y1, data=base_house)
    plt.title(f'Boxplot {variavel}')
    plt.xlabel(variavel)
    plt.ylabel('Price')
plt.tight_layout()
plt.show()
#É possivel observar que, embora haja outliers em todos os casos,somente quando o grafico é traçado usando os zipcodes há outliers negativos

In [ ]:
#Graficos boxplot para definição de outliers
X2=base_house["zipcode"]
Y2=base_house["price"]
plt.figure(figsize=(12, 6))
sns.boxplot(x=X2,y=Y2,data=base_house)
plt.show()

In [17]:
#Criando um dataframe somente com as casas que são outliers negativos dentro do seu zipcode

# Selecionar apenas as variáveis necessárias
outliers_negativos = base_house[['id','zipcode', 'price']]
# Calcular os limites para outliers por "zipcode" usando o método IQR
def calcular_limites(group):
    Q1 = group['price'].quantile(0.25)
    Q3 = group['price'].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    return group[group['price'] < limite_inferior]
# Aplicar a função para cada grupo (zipcode)
outliers_negativos_agrupados = outliers_negativos.groupby('zipcode').apply(calcular_limites)
# Resetar o índice após a operação de agrupamento
outliers_negativos_agrupados.reset_index(drop=True, inplace=True)

In [ ]:
#Traçar uma matriz de correlação entre as variaveis quantitativas e o preço ()
matriz_correlacao = variaveis_quantitativas.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.show()
#Analisando o gráfico é possivel observar que as variáveis que mais possuem correlação com a variável preço são: sqft_living,grade,sqft_above

## Levantamento de Hipóteses e Análise Exploratória de Dados

### Há uma diferença de preço, estatisticamente relevante, entre os zipcodes?

In [ ]:
#Teste de Hipoteses para testar se há uma diferença de preço estatisticamente relevante na média dos preços das casas entre zipcodes
#Foi utilizado o teste de wilconxon
#Hipoteses nula: Não há diferença no preços entre zipcodes
#obs:foi necessário colher amostras aleatórias para os zipcodes com mais de 50 casas para que todas as bases de comparação tivessem o mesmo tamanho

# Obter a lista única de códigos postais
codigos_postais_unicos = base_house['zipcode'].unique()
# Criar dicionário para armazenar as bases de dados separadas
bases_separadas = {}
# Loop para criar bases de dados separadas
for codigo_postal in codigos_postais_unicos:
    nome_base = f'base_zipcode_{codigo_postal}'
    base_temp = base_house.loc[base_house['zipcode'] == codigo_postal, ['price']].copy()
    # Excluir aleatoriamente linhas até que a base tenha 50 linhas
    while len(base_temp) > 50:
        linha_excluir = base_temp.sample(1).index
        base_temp.drop(linha_excluir, inplace=True)
    bases_separadas[nome_base] = base_temp


from scipy.stats import wilcoxon
from itertools import combinations
# Lista para armazenar os resultados do teste de Wilcoxon
resultados_wilcoxon = []
# Loop para realizar o teste de Wilcoxon para cada par de bases
for (nome_base1, base1), (nome_base2, base2) in combinations(bases_separadas.items(), 2):
    # Realizar o teste de Wilcoxon
    estatistica, p_valor = wilcoxon(base1['price'], base2['price'])
    # Armazenar os resultados
    resultados_wilcoxon.append({
        'Par de Bases': (nome_base1, nome_base2),
        'Estatística de Wilcoxon': estatistica,
        'P-valor': p_valor
    })

In [ ]:
#Conta a quantidade de pares que possuem valor de pvalue menor que 0,05 e portanto quantiadede de pares de zipcodes em que a hipotese nula pode ser descartada
resultados_wilcoxon = pd.DataFrame(resultados_wilcoxon)
(resultados_wilcoxon['P-valor'] < 0.05).sum()


In [ ]:
#Conta a quantidade de pares que possuem valor de pvalue maior  que 0,05 e portanto quantiadede de pares de zipcodes em que a hipotese nula não pode ser descartada
(resultados_wilcoxon['P-valor'] >= 0.05).sum()
#É possivel observar dessa forma, que há, entre a grande maioria dos zipcodes, uma diferença na média dos preços. Dessa forma, a escolha do zipcode influencia diretamente no preço.

### Quanto mais banheiros mais cara é a casa?


De acordo com a matriz de correlação traçada anteriormente é possivel afirmar que sim.
É possivel analisar, adicionalmente, a partir de e/ou até quantos banheiros o preço aumenta e qual o incremento de preço por cada banheiro.

In [ ]:
#Observando as estatisticas descritivas do preço por quantidade de banheiro
base_house.groupby('bathrooms')['price'].describe()
#Para essa analise será considerado somente a quantidade de banheiro entre 1 e 4, devido a quantidade irrelevante de casas fora dessa faixa

In [ ]:
#Grafico de evolução do preço em relação a quantidade de banheiros
from matplotlib.ticker import FuncFormatter
base_house_filtrada = base_house[(base_house['bathrooms'] >= 1) & (base_house['bathrooms'] <= 4)]
def milhares_de_dolares_formatter(x, pos):
    return f"${int(x/1000)}K"
plt.figure(figsize=(12, 6))
sns.barplot(x='bathrooms', y='price', data=base_house_filtrada, ci=None)
plt.xlabel('Número de Banheiros')
plt.ylabel('Média de Preço')
plt.title('Média de Preço por Número de Banheiros')
formatter = FuncFormatter(milhares_de_dolares_formatter)
plt.gca().yaxis.set_major_formatter(formatter)
plt.show()
#Pelo grafico foi levantada a hipotese que a partir de 2 banheiros o preço da casa começa a mudar. Para confirmar isso será feito o teste de hipotese para comparar a média do preço de casas com 1, 2, 3 e 4 ban

In [44]:
# Obter a lista única banheiros
base_house_filtrada = base_house_filtrada[base_house_filtrada['bathrooms'] != 1.25].copy()
banheiros_unicos = base_house_filtrada['bathrooms'].unique()
# Criar dicionário para armazenar as bases de dados separadas
bases_separadas1 = {}
# Loop para criar bases de dados separadas
for banheiros in banheiros_unicos:
    nome_base1 = f'base_banheiro_{banheiros}'
    base_temp1 = base_house_filtrada.loc[base_house_filtrada['bathrooms'] == banheiros, ['price']].copy()
    # Excluir aleatoriamente linhas até que a base tenha 136 linhas
    while len(base_temp1) > 136:
        linha_excluir1 = base_temp1.sample(1).index
        base_temp1.drop(linha_excluir1, inplace=True)
    bases_separadas1[nome_base1] = base_temp1

In [ ]:
# Lista para armazenar os resultados do teste de Wilcoxon
resultados_wilcoxon1 = []
# Loop para realizar o teste de Wilcoxon para cada par de bases
for (nome_base1, base1), (nome_base2, base2) in combinations(bases_separadas1.items(), 2):
    # Realizar o teste de Wilcoxon
    estatistica, p_valor = wilcoxon(base1['price'], base2['price'])
    # Armazenar os resultados
    resultados_wilcoxon1.append({
        'Par de Bases': (nome_base1, nome_base2),
        'Estatística de Wilcoxon': estatistica,
        'P-valor': p_valor
    })
resultados_wilcoxon1 = pd.DataFrame(resultados_wilcoxon1)
resultados_wilcoxon1

In [ ]:
resultados_wilcoxon1_filtrado = resultados_wilcoxon1.loc[resultados_wilcoxon1['P-valor'] > 0.05, :]
resultados_wilcoxon1_filtrado
# Por meio desses resultados é possivel afirmar que o acrescimo de 1 banheiro valoriza a casa. Mas o acresimo de 0,5 ou menos banheiros depende da quantidade
# Dessa forma, será feito o calculo do incremento de valor no acresimo de 1

In [ ]:
# Calculo do incremento
# Esse calculo será feito de acordo com o tamanho da casa e o zipcode, já que isso não pode ser alterado em uma possivel reforma e tem correlação com o preço.
valores_bathrooms = [1, 2, 3, 4]
percentis = [i / 10 for i in range(11)]
base_house_filtrada1 = base_house_filtrada[base_house_filtrada['bathrooms'].isin(valores_bathrooms)].copy()
base_house_filtrada1['sqft_living_percentil'] = pd.qcut(base_house_filtrada1['sqft_living'], q=percentis, labels=False)
preco_banheiro = base_house_filtrada1.groupby(['bathrooms', 'sqft_living_percentil','zipcode'])['price'].mean().reset_index()
preco_banheiro.dropna(subset=['price'], inplace=True)
preco_banheiro = preco_banheiro.sort_values(by=['zipcode','sqft_living_percentil'])
preco_banheiro['diferenca_preco'] = preco_banheiro['price'].diff()
preco_banheiro['teste'] = ((preco_banheiro['bathrooms'].shift(1) == preco_banheiro['bathrooms'] - 1) &
                                 (preco_banheiro['sqft_living_percentil'].shift(1) == preco_banheiro['sqft_living_percentil'])
                                     & (preco_banheiro['zipcode'].shift(1) == preco_banheiro['zipcode'])).astype(int)
preco_banheiro = preco_banheiro[preco_banheiro['teste'] == 1].copy()
preco_banheiro = preco_banheiro.groupby(['bathrooms'])['diferenca_preco'].mean().reset_index()
preco_banheiro
#Esses são os valores incrementais de adicionar 1 banheiro a casa. É possivel observar que a diferença de valor de uma casa de 3 para 4 banheiros com um tamanho similar é grande
#O calculo foi feito da seguinte forma: 1. Dividir a base em percentis de acodo com o tamanho da casa 2. Calcular a média do preço agrupando por qtde de banheiro, tamanho e zipcode
#3. Calcular a diferença média de preço de uma casa com 1 e 2 banheiros, por exemplo - entre uma casa na mesma região na mesma faixa de tamanho. 4. Calcular a média de aumento de preço com o incremento de um banheiro por quantidade de banheiro já existente

### Casas renovadas são mais caras?

In [48]:
# Criar dicionário para armazenar as bases de dados separadas
renovada_unica = base_house['renovated'].unique()
bases_separadas2 = {}
# Loop para criar bases de dados separadas
for renovada in renovada_unica:
    nome_base2 = f'base_renovada_{renovada}'
    base_temp2 = base_house.loc[base_house['renovated'] == renovada, ['price']].copy()
    # Excluir aleatoriamente linhas até que a base tenha 914 linhas
    while len(base_temp2) > 914:
        linha_excluir2 = base_temp2.sample(1).index
        base_temp2.drop(linha_excluir2, inplace=True)
    bases_separadas2[nome_base2] = base_temp2

In [ ]:
from scipy.stats import wilcoxon
from itertools import combinations
resultados_wilcoxon3 = []
# Loop para realizar o teste de Wilcoxon para cada par de bases
for (nome_base1, base1), (nome_base2, base2) in combinations(bases_separadas2.items(), 2):
    # Realizar o teste de Wilcoxon
    estatistica, p_valor = wilcoxon(base1['price'], base2['price'])
    # Armazenar os resultados
    resultados_wilcoxon3.append({
        'Par de Bases': (nome_base1, nome_base2),
        'Estatística de Wilcoxon': estatistica,
        'P-valor': p_valor
    })
resultados_wilcoxon3
# É possivel concluir que há uma diferença na média de preços entre casas renovadas ou não

In [ ]:
renovada = base_house.groupby(['renovated'])['price'].mean().reset_index()
renovada
#A casa renovada é mais cara que a casa não renovada


In [ ]:
# Calculo do incremento
# Esse calculo será feito de acordo com o tamanho da casa e o zipcode, já que isso não pode ser alterado em uma possivel reforma e tem correlação com o preço.
percentis = [i / 10 for i in range(11)]
base_house1=base_house
base_house1['sqft_living_percentil'] = pd.qcut(base_house1['sqft_living'], q=percentis, labels=False)
preco_renovada = base_house1.groupby(['renovated', 'sqft_living_percentil','zipcode'])['price'].mean().reset_index()
preco_renovada.dropna(subset=['price'], inplace=True)
preco_renovada = preco_renovada.sort_values(by=['zipcode','sqft_living_percentil'])
preco_renovada['diferenca_preco'] = preco_renovada['price'].diff()
preco_renovada['teste'] = ((preco_renovada['renovated'].shift(1) != preco_renovada['renovated']) &
                                 (preco_renovada['sqft_living_percentil'].shift(1) == preco_renovada['sqft_living_percentil'])
                                     & (preco_renovada['zipcode'].shift(1) == preco_renovada['zipcode'])).astype(int)
preco_renovada = preco_renovada[preco_renovada['teste'] == 1].copy()
preco_renovada = preco_renovada.groupby(['renovated'])['diferenca_preco'].mean().reset_index()
preco_renovada
#Esses são os valores incrementais de renovar a casa.

### Casas em melhor condição são mais cara?

In [ ]:
#Grafico de evolução do preço em relação a condição da casa
from matplotlib.ticker import FuncFormatter
base_house2 = base_house
def milhares_de_dolares_formatter(x, pos):
    return f"${int(x/1000)}K"
plt.figure(figsize=(12, 6))
sns.barplot(x='condition', y='price', data=base_house2, ci=None)
plt.xlabel('Condição')
plt.ylabel('Média de Preço')
plt.title('Média de Preço por Condição da Casa')
formatter = FuncFormatter(milhares_de_dolares_formatter)
plt.gca().yaxis.set_major_formatter(formatter)
plt.show()
#Pelo grafico levanta-se a hipotese de que o preço médio das casas aumenta, somente, quando elas saem de uma condição 2 para 3.

In [53]:
# Obter a lista única de condição
#Tirando casas com condição = 1 devido a quantidade reduzida de casas nessa condição
base_house2 = base_house[base_house['condition'] != 1].copy()
condition_unicos = base_house2['condition'].unique()
# Criar dicionário para armazenar as bases de dados separadas
bases_separadas3 = {}
# Loop para criar bases de dados separadas
for condition in condition_unicos:
    nome_base3 = f'base_zipcode_{condition}'
    base_temp3 = base_house2.loc[base_house2['condition'] == condition, ['price']].copy()
    # Excluir aleatoriamente linhas até que a base tenha 50 linhas
    while len(base_temp3) > 172:
        linha_excluir3 = base_temp3.sample(1).index
        base_temp3.drop(linha_excluir3, inplace=True)
    bases_separadas3[nome_base3] = base_temp3

In [ ]:
# Lista para armazenar os resultados do teste de Wilcoxon
resultados_wilcoxon2 = []
# Loop para realizar o teste de Wilcoxon para cada par de bases
for (nome_base1, base1), (nome_base2, base2) in combinations(bases_separadas3.items(), 2):
    # Realizar o teste de Wilcoxon
    estatistica, p_valor = wilcoxon(base1['price'], base2['price'])
    # Armazenar os resultados
    resultados_wilcoxon2.append({
        'Par de Bases': (nome_base1, nome_base2),
        'Estatística de Wilcoxon': estatistica,
        'P-valor': p_valor
    })
resultados_wilcoxon2 = pd.DataFrame(resultados_wilcoxon2)
resultados_wilcoxon2
#Conforme previsto há uma diferença estatisticamente significante na media dos preços quando a casa sai do patamar de 2 para 3 na condição

In [ ]:
# Calculo do incremento
# Esse calculo será feito de acordo com o tamanho da casa e o zipcode, já que isso não pode ser alterado em uma possivel reforma e tem correlação com o preço.
percentis = [i / 10 for i in range(11)]
base_house2['sqft_living_percentil'] = pd.qcut(base_house2['sqft_living'], q=percentis, labels=False)
base_house2['condition'] = base_house2['condition'].astype(int)
preco_condicao = base_house2.groupby(['condition', 'sqft_living_percentil','zipcode'])['price'].mean().reset_index()
preco_condicao.dropna(subset=['price'], inplace=True)
preco_condicao = preco_condicao.sort_values(by=['zipcode','sqft_living_percentil'])
preco_condicao['diferenca_preco'] = preco_condicao['price'].diff()
preco_condicao['teste'] = ((preco_condicao['condition'].shift(1) == preco_condicao['condition'] - 1) &
                                 (preco_condicao['sqft_living_percentil'].shift(1) == preco_condicao['sqft_living_percentil'])
                                     & (preco_condicao['zipcode'].shift(1) == preco_condicao['zipcode'])).astype(int)
preco_condicao = preco_condicao[preco_condicao['teste'] == 1].copy()
preco_condicao = preco_condicao.groupby(['condition'])['diferenca_preco'].mean().reset_index()
preco_condicao = preco_condicao[preco_condicao['condition'] == 3].copy()
preco_condicao

## Insights e Respostas

**Insights**



1.   Não há nenhuma variável que sozinha tenha uma correlação considerada muito forte com o preço da casa. Porém, o tamanho da casa e o estado em que ela se encontra possuem uma relação relativamente forte.
2.   A única variavel que produz outliers negativos relacionados ao preço é o zipcode. Esses outliers negativos podem ser usados para compra da casa em um preço abaixo do mercado. É necessario tomara cuidado com o preço de venda pois há uma diferença estatisticamente significante do preço médio das casas dependendo do zipcode.
3.   O acresimo de 1 banheiro valoriza a casa, mesmo quando mantido o tamanho da mesma.
4.   Casas em  renovadas tem um preço estatiticamente maior que casas não renovadas e com uma avaliação ruim. Há uma diferença de preço, estatisticamente significante, quando se compara uma casa com nota de condição igual ou abaixo de 2 com uma casa com nota acima de 2.



In [ ]:
#Definição da sugestão de casas que devem ser compradas
outliers_negativos_agrupados = pd.merge(outliers_negativos_agrupados, base_house[['id', 'renovated', 'condition','bathrooms']], on='id', how='left')
outliers_negativos_agrupados['condition'] = outliers_negativos_agrupados['condition'].astype('int')
filtro  = (outliers_negativos_agrupados['renovated'] == '0') & (outliers_negativos_agrupados['condition'] <= 2)
outliers_negativos_agrupados = outliers_negativos_agrupados[filtro]
#filtrando somente casas que são outliers negativos em sua respectiva localização, que tenha uma condição <=2 e que não tenha sido reformada.
outliers_negativos_agrupados


In [63]:
#Definição do preço de revenda das casas
outliers_negativos_agrupados_preco=outliers_negativos_agrupados
outliers_negativos_agrupados_preco = outliers_negativos_agrupados_preco.drop(['zipcode', 'renovated', 'condition', 'bathrooms'], axis=1)
valor_referente_bathrooms_2 = preco_banheiro.loc[preco_banheiro['bathrooms'] == 2, 'diferenca_preco'].values[0]
outliers_negativos_agrupados_preco['valor_referente_bathrooms'] = valor_referente_bathrooms_2
valor_referente_condicao = preco_condicao.loc[preco_condicao['condition'] == 3, 'diferenca_preco'].values[0]
outliers_negativos_agrupados_preco['valor_referente_condition'] = valor_referente_condicao
valor_referente_reforma = preco_renovada.loc[preco_renovada['renovated'] == '1', 'diferenca_preco'].values[0]
outliers_negativos_agrupados_preco['valor_referente_renovated'] = valor_referente_reforma
outliers_negativos_agrupados_preco['preco_venda_final'] = outliers_negativos_agrupados_preco['price'] + outliers_negativos_agrupados_preco['valor_referente_bathrooms'] + outliers_negativos_agrupados_preco['valor_referente_condition'] + outliers_negativos_agrupados_preco['valor_referente_renovated']
outliers_negativos_agrupados_preco

**Respostas**

1.   As casas que podem despertar o interesse do CEO para aquisição são classificadas como outliers negativos, destacando-se por terem preços de venda inferiores ou iguais a Q1 - 1,5 * IQR¹. Essa categorização sugere uma oportunidade de valorização, indicando que tais propriedades estão significativamente abaixo da mediana de preços para a localização. Além disso, a seleção é restrita a casas que não passaram por processos de renovação e apresentam uma condição considerada ruim, o que também representa uma oportunidade de valorização. O preço de compra reflete a listagem atual no mercado, sugerindo que estão disponíveis por um valor abaixo da média. O CEO está interessado em explorar essas condições de mercado, considerando essas propriedades como investimentos estratégicos, buscando potenciais ganhos de valorização no futuro. As informações específicas sobre essas residências estão contidas no DataFrame 'outliers_negativos_agrupados'.
2.   O momento mais propício para a revenda das casas ocorre após a conclusão da reforma. O preço de venda resultante é calculado somando o valor inicial de compra ao incremento de preço associado às melhorias realizadas durante a reforma. A reforma pode focar em aumentar o nivel das casas para pelo menos uma condição 3. Adicionalmente, a adição de um banheiro durante a reforma pode significativamente elevar o valor de revenda. O preço de revenda das casas selecionadas pode ser encontrado no DataFrame 'outliers_negativos_agrupados_preco'.
3.   Conforme destacado na questão anterior, a realização de uma renovação na casa não apenas eleva o preço de revenda, mas também possibilita melhorias significativas na condição da propriedade, contribuindo assim para um aumento no valor final. Além disso, identificou-se que a quantidade de banheiros é a única variável, além de condição e renovação, que apresenta uma correlação substancial com o preço de revenda. Diante desse insight, sugere-se a consideração do acréscimo de um banheiro às casas, como uma estratégia para impulsionar o preço de revenda. Para avaliar os valores específicos de incremento para as casas com essa recomendação, é possível consultar o DataFrame 'outliers_negativos_agrupados_preco'.

¹IQR=Diferença Inter Quartil: Q3-Q1